In [9]:
import pandas as pd
import pickle
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [19]:
def table_parser(page):
    file = open(page)
    # page = page.split('\n')
    table = []
    num = 0
    for line in file:
        if 'Grade' in line:
            num += 1
        if num > 0:
            num += 1
        if 3 <= num < 21:
            line = line.rstrip()
            if line != '':
                split_line = line.split(' ')
                split_line = [x for x in split_line if x != '']
                strip_line = split_line[:16]
                table.append(strip_line)
    WG = []
    WL = []
    WS = []
    for l in table:
    # WG.append(l[0:1])
    # WL.append(l[0:1])
    # WS.append(l[0:1])
        WG.append((l[1:6]))
        WL.append(l[6:11])
        WS.append(l[11:16])

    file.close()
    return pd.DataFrame(WG,columns=['Step 1','Step 2','Step 3','Step 4','Step 5'], index=['WG 1','WG 2','WG 3','WG 4','WG 5','WG 6','WG 7','WG 8','WG 9','WG 10','WG 11','WG 12','WG 13','WG 14','WG 15']), pd.DataFrame(WL,columns=['Step 1','Step 2','Step 3','Step 4','Step 5'], index=['WG 1','WG 2','WG 3','WG 4','WG 5','WG 6','WG 7','WG 8','WG 9','WG 10','WG 11','WG 12','WG 13','WG 14','WG 15']), pd.DataFrame(WS,columns=['Step 1','Step 2','Step 3','Step 4','Step 5'], index=['WG 1','WG 2','WG 3','WG 4','WG 5','WG 6','WG 7','WG 8','WG 9','WG 10','WG 11','WG 12','WG 13','WG 14','WG 15'])

In [20]:
test_df = pd.concat(table_parser('AF Schedule Area 077R Northern Mississippi (RUS) Effective_ 17 April 2016.html'))

In [21]:
test_df

,Step 1,Step 2,Step 3,Step 4,Step 5
WG 1,11.03,11.50,11.93,12.42,12.85
WG 2,11.95,12.46,12.93,13.46,13.96
WG 3,12.87,13.41,13.96,14.46,15.02
WG 4,13.76,14.36,14.90,15.51,16.05
WG 5,14.67,15.25,15.88,16.48,17.10
WG 6,15.58,16.21,16.86,17.50,18.15
WG 7,16.49,17.17,17.88,18.55,19.24
WG 8,17.47,18.16,18.91,19.63,20.37
WG 9,18.34,19.09,19.88,20.67,21.42
WG 10,19.14,19.99,20.84,21.62,22.44
